<a href="https://colab.research.google.com/github/tthogho1/CompareImage/blob/main/stable_diffusion_flask2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Mar 21 10:47:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    33W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install diffusers transformers ftfy accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.1 MB/s eta 0:00:00


In [2]:
!pip install flask-ngrok
!pip install flask_cors
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 31.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19790 sha256=c69f2a6cec24d566aca4777bd0d9fd432c3acc4a9df7c31ff2ef02412bf4b1af
  Stored in directory: /root/.cache/pip/wheels/f6/89/59/49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


In [3]:
!ngrok authtoken 1gOMCWXlgHXaeMbnoxbkiCE1sUM_3Y88VCBao4whAAUc6tBnP

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
from flask import Flask,request,make_response
from flask_ngrok import run_with_ngrok
import torch
from diffusers import StableDiffusionPipeline
import base64
from io import BytesIO
from flask_cors import CORS

model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

#prompt = "Mt. Fuji in the style of Gauguin"
pipe = StableDiffusionPipeline.from_pretrained(model_id, revision="fp16", torch_dtype=torch.float16)
pipe = pipe.to(device)

@app.route("/" ,methods=['GET','POST'])
def home():
	if request.method == 'GET':
		return "home"

	prompt = request.form['prompt']
	print("prompt:" + prompt )
	generator = torch.Generator(device).manual_seed(0) # 
	image = pipe(prompt, guidance_scale=7.5, generator=generator).images[0]

	# 生成した画像の保存
	jpeg_data = BytesIO()
	image.save(jpeg_data, format='JPEG')
	image_bytes = jpeg_data.getvalue()
	base64_string = base64.b64encode(image_bytes).decode('utf-8')

	image_string = "data:image/jpeg;base64," + base64_string
	response = make_response(image_string, 200)
	response.mimetype = "text/plain"
	return response

app.run()


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://6443-34-90-223-207.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
